In [947]:
from __future__ import print_function
import numpy as np
from collections import Counter
import itertools
from tqdm import tqdm

enable_print = 0


def print(*args, **kwargs):
    if enable_print:
        return __builtins__.print(*args, **kwargs)


enable_print = 1  #toggle to turn off printing of logs


class agent(object):
    #-----------------------------------------------------------------------------------------------#
    '''
    Agent Generation Mechanism. Extracts node attribute information from graph structure

    PARMS:
        node_number: int, assigning variable name to agents. Just use index position
        G: A networkx graph structure; G(V,E)
        env: an environment. Environment and Agents will communicate and pass back and forth state information
        seed: int, used for random number generator

                                env(G(V,E), Agents(G(V,E), env))
                                agents operate within the environment class.

    RETURNS:
        Graph with node attributes set
    '''

    #-----------------------------------------------------------------------------------------------#
    def __init__(self, node_number, G, env, seed=1, enable_print=1):
        self.node_number = node_number
        self.nodeData = env.nodeData                                 #key (node number): (strategy, p_act, neighbors, degree)
        self.seed = np.random.seed(seed)
        self.letters = {
            'letters_initial': [],
            'letters_stolen': [],
            'letters_received': []
        }
        self.target_word = []
        self.l_union_vi = []
        self.letters_needed = []
        self.corpus = []
        self.actions_taken = 0 
        self.words_formed = 0

    def _merge_letters(self):
        #-----------------------------------------------------------------------------------------------#
        '''
        Merge all letters in agent.letters 
        This returns a set of all letters, including non-unique entries
        '''
        #-----------------------------------------------------------------------------------------------#
        self.l_union_vi = list(itertools.chain.from_iterable(self.letters.values()))

    def _find_needed_letters(self):
        #-----------------------------------------------------------------------------------------------#
        '''
        Find the letters needed to complete a target word. Calc difference between two independent sets.
        '''
        #-----------------------------------------------------------------------------------------------#
        self._merge_letters()
        self.letters_needed = np.setdiff1d(self.target_word, self.l_union_vi)
        
    def _update_state_info(self,env): 
        self._merge_letters() #Remerge and update lunion
        self._find_needed_letters() #re-find needed letters
        env.historical_states[env.time] = env.current_state
        env.current_state[self.node_number] = self.letters
        env.time += 1
        self.actions_taken += 1
        print('state info updated')

    def get_init_hand(self, env):
        #-----------------------------------------------------------------------------------------------#
        '''
        Get initial hand prior to game start. 
        t=0 at this point. We randomly choose letters over the alphabet uniformly.
        Assign back to dict in environment containing local state information.
        '''
        #-----------------------------------------------------------------------------------------------#
        self.letters['letters_initial'] = np.random.choice(
            env.alphabet, 5)  #k is the length of each element in corpus
        env.current_state[self.node_number] = self.letters

    def get_target_word(self, env):
        #-----------------------------------------------------------------------------------------------#
        '''
        After c^possible is instantiated and set by the env, we use an augmented form of hamming distance 
        to determine a target word that is the closest to us. 
        '''
        #-----------------------------------------------------------------------------------------------#
        matches = []
        for i in self.corpus:
            matches.append(
                len(self.l_union_vi) -
                len([j for j in self.l_union_vi if j in i]))
        tword = self.corpus[np.argmin(matches)]
        self.corpus.remove(tword)
        self.target_word = [i for i in tword]
        self._find_needed_letters()

    def _agent_form_word(self,env):
        #-----------------------------------------------------------------------------------------------#
        '''
        Need to update to remove letters from Lrecieved and Lstolen if used to form a word
        '''
        #-----------------------------------------------------------------------------------------------#
        assert env.time < env.time_max, 'Maximum time allowed has been reached'
        if len(self.letters_needed) == 0:
            env.words_formed.append(("".join(self.target_word),self.node_number)) #global counter
            self.get_target_word(env) #get new target word
        else:
            print('unable to form word. Need additional letters.')
        self._update_state_info(env)
        
        

    def _agent_steal_letter(self,env):
        #-----------------------------------------------------------------------------------------------#
        '''
        Find letters needed to complete the target word. 
        Randomly selects a single letter.
        Finds neighbors of current node and scans their union set to see if this letter exists. If it does, steals letter.
        Update env.current state and pop current state to historical state. Increment timer
        Need to update so a letter stolen from non init dist is depleted
        '''
        #-----------------------------------------------------------------------------------------------#
        assert env.time < env.time_max, 'Maximum time allowed has been reached'
        print('action to node: {}'.format(self.node_number))
        print('taking action: steal letter')
        neighbors = self.nodeData[self.node_number][2]
        print('available neighbors: {}'.format(neighbors))
        letters_needed = self.letters_needed
        print(f'letters needed to reach the target {letters_needed}')
        if len(letters_needed) != 0: 
            choose_letter_to_steal = np.random.choice(letters_needed)
        else: choose_letter_to_steal = None
        
        if choose_letter_to_steal != None:
            print(f'randomly selecting letter to steal: {choose_letter_to_steal}')
            for k, v in env.agents.items():
                if k in neighbors:
                    if choose_letter_to_steal in v.l_union_vi:
                        print(
                            f'neighbor {env.agents[k].node_number} : neighbor hand {env.agents[k].l_union_vi}'
                        )
                        self.letters['letters_stolen'].append(
                            choose_letter_to_steal)  #update local state
                        print('letter successfully stolen from node {}'.format(v))
                        break
        else:
            print('unable to steal the chosen letter from k-hop neighbors')

        self._update_state_info(env)

    def _agent_pass_letter(self,env):
        #-----------------------------------------------------------------------------------------------#
        '''
        Need to update so a letter passed from non init dist is depleted
        '''
        #-----------------------------------------------------------------------------------------------#
        assert env.time < env.time_max, 'Maximum time allowed has been reached'
        print('action to node: {}'.format(self.node_number))
        print('taking action: pass letter')
        neighbors = self.nodeData[self.node_number][2]
        print('available neighbors: {}'.format(neighbors))
        choose_letter_to_pass = np.random.choice(self.letters['letters_initial'])
        if choose_letter_to_pass != None:
            print(f'randomly selecting letter to pass: {choose_letter_to_pass}')
            for k, v in env.agents.items():
                if k in neighbors:
                    if choose_letter_to_pass not in v.l_union_vi:
                        print(
                            f'neighbor {env.agents[k].node_number} : neighbor hand {env.agents[k].l_union_vi}'
                        )
                        env.agents[k].letters['letters_received'].append(
                            choose_letter_to_pass)  #update local state
                        print('letter successfully passed from node {} to node {}'.format(self.node_number, v))
                        break
        else:
            print('unable to steal the chosen letter from k-hop neighbors')

        self._update_state_info(env)

    def _agent_think(self,env):
        #-----------------------------------------------------------------------------------------------#
        '''
        '''
        #-----------------------------------------------------------------------------------------------#
        assert env.time < env.time_max, 'Maximum time allowed has been reached'
        print('action to node: {}'.format(self.node_number))
        print('taking action: think')
        self._update_state_info(env)

    def take_action(self,env):
        #-----------------------------------------------------------------------------------------------#
        '''
        '''
        #-----------------------------------------------------------------------------------------------#
        n_Data = self.nodeData[self.node_number]
        node = self.node_number
        strategy = n_Data[0]
        p_act = n_Data[1]
        neighbors = n_Data[2]
        action_sampling = np.random.choice(env.action_space, p=p_act)

        if action_sampling == env.action_space[0]:  #form_word
            self._agent_form_word(env)
        if action_sampling == env.action_space[1]:  #steal_letter
            self._agent_steal_letter(env)
        if action_sampling == env.action_space[2]:  #pass_letter
            self._agent_pass_letter(env)
        if action_sampling == env.action_space[3]:  #think / null action
            self._agent_think(env)

In [948]:
from utils.helpers import possible_words


class environment(object):
    #-----------------------------------------------------------------------------------------------#
    '''
    Agent Generation Mechanism. Extracts node attribute information from graph structure

    PARMS:
        G: A networkx graph structure; G(V,E)

    '''
    #-----------------------------------------------------------------------------------------------#
    def __init__(self, G, t_max):

        self.G = G
        self.nodes = G.nodes
        self.edges = G.edges
        self.agents = []
        self.action_space = [
            'form_word', 'steal_letter', 'pass_letter', 'think'
        ]
        self.l_init_union = []
        self.corpus = np.loadtxt(fname='txt/five_letter_words.txt',
                                 dtype='str')
        self.corpus_possible = []
        self.alphabet = np.loadtxt(fname='txt/alphabet_english.txt',
                                   dtype='str')
        self.nodeData = {}
        self.current_state = {}  #at t
        self.historical_states = {}  #pop from current state at t+1
        self.words_formed = []  #by player
        self.time = 0
        self.time_max = t_max

    def _getnodeData(self):
        #-----------------------------------------------------------------------------------------------#
        '''
        '''
        #-----------------------------------------------------------------------------------------------#
        for i, j in enumerate(self.G.nodes):
            self.nodeData[i] = (self.G.nodes[i]['atts'][0],
                                self.G.nodes[i]['atts'][1],
                                [j for j in self.G.neighbors(i)],
                                self.G.degree(i))

    def _find_cand_words(self):
        #-----------------------------------------------------------------------------------------------#
        '''
        '''
        #-----------------------------------------------------------------------------------------------#
        l_init_union = [[i for l in d.values() for i in l]
                        for d in self.current_state.values()
                        ]  #Compute Union over all init letter dist
        self.l_init_union = list(
            itertools.chain(*l_init_union))  #flatten to 1d array

        self.corpus_possible = possible_words(
            self.corpus, self.l_init_union)  #find C^possible
        
        for i in self.agents:
            self.agents[i].corpus = self.corpus_possible

        print(
            f'total corpus size {len(self.corpus)}, total count of possible words {len(self.corpus_possible)}'
        )

    def _push_to_historical(self):
        #-----------------------------------------------------------------------------------------------#
        '''
        '''
        #-----------------------------------------------------------------------------------------------#
        self.historical_states[self.time] = self.current_state

    def _set_init_targets(self):
        #-----------------------------------------------------------------------------------------------#
        '''
        '''
        #-----------------------------------------------------------------------------------------------#
        [
            self.agents[i].get_target_word(self)
            for i in range(len(self.G.nodes))
        ]
        self._push_to_historical()  #push to historical states dict
        self.time += 1  #increment time counter

    def set_agents(self):
        #-----------------------------------------------------------------------------------------------#
        '''
        '''
        #-----------------------------------------------------------------------------------------------#
        #-------------------------------------------------
        #Create Agents
        #-------------------------------------------------
        agents = {}
        num_nodes = len(self.G.nodes)
        agent_names = [str(i) for i in range(len(self.G.nodes))]
        for i, j in enumerate(agent_names):
            agents[i] = agent(node_number=i, G=self.G, env=self)
        #-------------------------------------------------
        #Get init hand and pass back data to env.current_state
        #-------------------------------------------------
        print('agents passed into env')
        self.agents = agents

        [j.get_init_hand(self) for i, j in self.agents.items()
         ]  #-> L_init(agent) & env.current_state
        print('all agents assigned letters_initial')

        print('searching for candidate words C^possible in C')
        self._find_cand_words(
        )  #Find Candidate Words - C^Possible, given init letter distr

        print('agents passed into env')
        print('init target words set locally. Time counter incremented by 1')
        self._set_init_targets()  #Set initial target word

    def set_env(self):
        #-----------------------------------------------------------------------------------------------#
        '''
        '''
        #-----------------------------------------------------------------------------------------------#
        assert len(self.alphabet) == 26, 'Alphabet size mismatch'
        self._getnodeData()
        self.set_agents()
        print('Environment set - > Graph Created. Node attributes assigned.')
        
    def output_logs(self):
        pass
    
    def play(self):
        while self.time < self.time_max:
            for j in range(0, len(self.agents)):
                self.agents[j].take_action(self)
        

    def reset_env(self):
        #-----------------------------------------------------------------------------------------------#
        '''
        '''
        #-----------------------------------------------------------------------------------------------#
        pass

In [954]:
import os
from utils.graphinit import Network

#Graph Generation
num_nodes = 5
prob_edges = 0.5

#Node Attributes
pr_selfish = 0.5
pr_altrustic = 1 - pr_selfish
p_star = 0.20

G, table = Network(num_nodes, prob_edges, pr_selfish,
                  p_star).make_random_graph(plot=True, print_table=True)


env = environment(G, t_max= 500)
env.set_env()
env.play()

  node       strat                 p_act  neighbors degree
0   v0     selfish  [0.1, 0.4, 0.2, 0.3]        [4]      1
1   v1  altruistic  [0.1, 0.2, 0.4, 0.3]     [2, 4]      2
2   v2  altruistic  [0.1, 0.2, 0.4, 0.3]  [1, 3, 4]      3
3   v3  altruistic  [0.1, 0.2, 0.4, 0.3]        [2]      1
4   v4  altruistic  [0.1, 0.2, 0.4, 0.3]  [0, 1, 2]      3


agents passed into env
all agents assigned letters_initial
searching for candidate words C^possible in C
total corpus size 5757, total count of possible words 605
agents passed into env
init target words set locally. Time counter incremented by 1
Environment set - > Graph Created. Node attributes assigned.


  0%|          | 0/10000 [00:00<?, ?it/s]

action to node: 0
taking action: think
state info updated
action to node: 1
taking action: pass letter
available neighbors: [2, 4]
randomly selecting letter to pass: f
neighbor 2 : neighbor hand ['b', 'm', 'h', 'n', 'g']
letter successfully passed from node 1 to node <__main__.agent object at 0x0000024C8B102CC0>
state info updated
action to node: 2
taking action: steal letter
available neighbors: [1, 3, 4]
letters needed to reach the target ['i' 'o']
randomly selecting letter to steal: i
state info updated
action to node: 3
taking action: think
state info updated
action to node: 4
taking action: think
state info updated
action to node: 0
taking action: think
state info updated
unable to form word. Need additional letters.
state info updated
action to node: 2
taking action: pass letter
available neighbors: [1, 3, 4]
randomly selecting letter to pass: b
neighbor 1 : neighbor hand ['l', 'f', 'p', 'a', 'q']
letter successfully passed from node 2 to node <__main__.agent object at 0x0000024C

  0%|          | 6/10000 [00:00<02:56, 56.76it/s]

...

 12%|█▏        | 1208/10000 [00:21<02:37, 55.68it/s]

action to node: 0
taking action: steal letter
available neighbors: [4]
letters needed to reach the target []
unable to steal the chosen letter from k-hop neighbors
state info updated
action to node: 1
taking action: think
state info updated
action to node: 2
taking action: steal letter
available neighbors: [1, 3, 4]
letters needed to reach the target []
unable to steal the chosen letter from k-hop neighbors
state info updated
action to node: 3
taking action: think
state info updated
state info updated
action to node: 0
taking action: think
state info updated
action to node: 1
taking action: think
state info updated
action to node: 2
taking action: think
state info updated
action to node: 3
taking action: pass letter
available neighbors: [2]
randomly selecting letter to pass: s
state info updated
action to node: 4
taking action: steal letter
available neighbors: [0, 1, 2]
letters needed to reach the target []
unable to steal the chosen letter from k-hop neighbors
state info updated
acti

 12%|█▏        | 1214/10000 [00:21<02:35, 56.63it/s]

action to node: 0
taking action: think
state info updated
action to node: 1
taking action: pass letter
available neighbors: [2, 4]
randomly selecting letter to pass: f
state info updated
action to node: 2
taking action: pass letter
available neighbors: [1, 3, 4]
randomly selecting letter to pass: m
state info updated
action to node: 3
taking action: think
state info updated
action to node: 4
taking action: pass letter
available neighbors: [0, 1, 2]
randomly selecting letter to pass: s
state info updated
action to node: 0
taking action: steal letter
available neighbors: [4]
letters needed to reach the target []
unable to steal the chosen letter from k-hop neighbors
state info updated
action to node: 1
taking action: pass letter
available neighbors: [2, 4]
randomly selecting letter to pass: q
state info updated
action to node: 2
taking action: pass letter
available neighbors: [1, 3, 4]
randomly selecting letter to pass: h
state info updated
action to node: 3
taking action: think
state in

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [714]:
#!pip install dash
import dash
import dash_table
import pandas as pd

df = pd.DataFrame.from_dict(env.current_state)

df.T

letters_initial letters_received letters_stolen
0  [f, l, m, i, j]               []             []
1  [l, f, p, a, q]               []             []
2  [b, m, h, n, g]               []             []
3  [z, s, u, f, s]               []             []
4  [u, l, k, o, s]               []             []
5  [e, x, x, j, r]               []             []
6  [x, a, w, n, j]               []             []
7  [j, h, w, z, b]               []             []
8  [a, r, i, y, n]               []             []
9  [t, p, k, z, i]               []             []